In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 21 09:34:17 2023

@author: user
"""

# =============================================================================
# 
import os
import netCDF4 as nc
import pandas as pd

import numpy as np
import numpy.ma as ma

import xarray as xr

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.style as mplstyle

import cartopy.crs as ccrs
import cartopy.feature as cfeat
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.io.shapereader import Reader
import cartopy.io.shapereader as shpreader
from cartopy.mpl.ticker import LongitudeFormatter,LatitudeFormatter
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import netCDF4 as nc
from netCDF4 import Dataset

import shapefile
import time

# =============================================================================


In [ ]:
import cartopy.feature as cfeature
import cmaps
import matplotlib.colors as mcolors 
from matplotlib.patches import ConnectionPatch
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from netCDF4 import Dataset
from scipy.interpolate import griddata
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


def read_sample_nc(SM_path, SM_filename):
    # SM_filename = f"20170815.1km_{VAR}_000010.nc"
    SM_file = os.path.join(SM_path , SM_filename)
    nc_SM = Dataset(SM_file)
    SM=(nc_SM.variables['SM'][:])
    XLAT = (nc_SM.variables['lat'][:])
    XLON = (nc_SM.variables['lon'][:])
    SM[SM < 0] = np.nan
    nc_SM.close()
    return SM, XLAT, XLON


import matplotlib.ticker as mticker
from matplotlib.image import imread

def add_Lat_and_Lon_Grid(ax, Domain, xy_labelsize, XInterval, YInterval):
    # 移除网格线
    ax.gridlines(draw_labels=False, linewidth=0, alpha=0)
    
    # 设置经纬度刻度
    LATmin, Latmax, LONmin, LONmax = Domain
    lon_range = int(LONmax)-int(LONmin)
    lat_range = int(Latmax)-int(LATmin)
    # print(LATmin, Latmax, LONmin, LONmax)
    # 动态计算刻度间隔
    # lon_interval = max(1, int(lon_range / Interval))  # 假设最多12个刻度线
    # lat_interval = max(1, int(lat_range / Interval))  # 假设最多8个刻度线
    lon_interval = XInterval
    lat_interval = YInterval
    ax.set_xticks(np.arange(int(LONmin)+1, int(LONmax) + 1, lon_interval), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(int(LATmin)+1, int(Latmax) + 1, lat_interval), crs=ccrs.PlateCarree())

    # 设置刻度标签格式
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    
    # 设置刻度线样式
    ax.tick_params(axis='both', which='major', direction='out', length=1.5, width=0.5, color='black', 
                   labelsize=xy_labelsize, labelcolor='black', pad=2.5)
    
    # 只在底部和左侧显示刻度标签
    ax.xaxis.set_tick_params(labeltop=False, labelbottom=True)
    ax.yaxis.set_tick_params(labelleft=True, labelright=False)

    # 设置轴线颜色为黑色
    ax.spines['left'].set_color('black')
    ax.spines['bottom'].set_color('black')
    ax.spines['right'].set_color('black')
    ax.spines['top'].set_color('black')
    
from matplotlib import ticker

# -----------------------------------------------------------------
# 设置参数
MODEL_NAME = "CSMX"
# 设置路径

SM_path = "/home/yfdong/data/work/SMmerge/CN/merge/code/analysis_code/SpatialPattern/nc"
SM_filename = '20170815.CSMX_000010.nc'

# 设置绘图属性
text_size = 8
cbar_titlesize = 8
cbar_lablesize = 8
title_size = 8
legend_size = 8
xy_labelsize = 7 #x轴和轴上刻度字体大小
Rectangle_width =1.
SHRINK = 0.6
DEMcmap = cmaps.OceanLakeLandSnow
cmap = cmaps.MPL_StepSeq

In [ ]:
# ------------------------------------------------示例图---------------------------------------------------------
#-----------函数：添加指北针--------------
def add_north(ax, labelsize=10, loc_x=0.05, loc_y=0.98, width=0.05, height=0.11, pad=0.14):
    import matplotlib.patches as mpatches
    """
    画一个比例尺带'N'文字注释
    主要参数如下
    :param ax: 要画的坐标区域 Axes实例 plt.gca()获取即可
    :param labelsize: 显示'N'文字的大小
    :param loc_x: 以文字下部为中心的占整个ax横向比例
    :param loc_y: 以文字下部为中心的占整个ax纵向比例
    :param width: 指南针占ax比例宽度
    :param height: 指南针占ax比例高度
    :param pad: 文字符号占ax比例间隙
    :return: None
    """
    minx, maxx = ax.get_xlim()
    miny, maxy = ax.get_ylim()
    ylen = maxy - miny
    xlen = maxx - minx
    left = [minx + xlen*(loc_x - width*.5), miny + ylen*(loc_y - pad)]
    right = [minx + xlen*(loc_x + width*.5), miny + ylen*(loc_y - pad)]
    top = [minx + xlen*loc_x, miny + ylen*(loc_y - pad + height)]
    center = [minx + xlen*loc_x, left[1] + (top[1] - left[1])*.4]
    triangle = mpatches.Polygon([left, top, right, center], color='k')
    ax.text(s='N',
            x=minx + xlen*loc_x,
            y=miny + ylen*(loc_y - pad + height),
            fontsize=labelsize,
            horizontalalignment='center',
            verticalalignment='bottom')
    triangle1 = mpatches.Polygon([left, top, center], facecolor='black', edgecolor='black', linewidth=1)
    triangle2 = mpatches.Polygon([right, top, center], facecolor='white', edgecolor='black', linewidth=1)
    
    ax.add_patch(triangle1)
    ax.add_patch(triangle2)

# -------------------------------------绘制子图------------------------------------

SM, XLAT, XLON = read_sample_nc(SM_path, SM_filename)
# =============================================================================
# -----------------------------绘图-------------------------------------
proj = ccrs.PlateCarree()  # 创建坐标系
fig = plt.figure(figsize=(9*1.1, 7*1.1), dpi=500)  # 创建页面
ax = fig.subplots(1, 1, subplot_kw={'projection': proj})  
# # 设置绘图区背景颜色
ax.set_facecolor('#F5F5F5')
# 添加海洋淡蓝色填充，以及海岸线
ax.add_feature(cfeature.OCEAN, zorder=0, facecolor='lightblue')
ax.add_feature(cfeature.COASTLINE, zorder=0, edgecolor='black', linewidth=0.5)
imshows = ax.imshow(SM[:], cmap=cmap, extent=[XLON.min(), XLON.max(), XLAT.min(), XLAT.max()], origin='lower')
imshows.set_clim(0.0, 0.4)

# 添加 colorbar
cax = fig.add_axes([0.21, 0.7, 0.03, 0.05])  # [左, 下, 宽, 高]
cbar = fig.colorbar(imshows, cax=cax, orientation='vertical', ticks=[0, 0.4])
cbar.ax.set_title('SM (m³/m³)', fontsize=10)

#-----------------------------添加指北针-------------------------------
add_north(ax)  
Domain1=[17.5, 59, 73, 135]
add_Lat_and_Lon_Grid(ax, Domain1, xy_labelsize-0.5, XInterval=15, YInterval=10)
# =============================================================================
# -----------------------------添加南海小地图--------------------------------
# 设置南海小地图的位置和大小
left, bottom, width, height = 0.725, 0.21, 0.22*0.9, 0.25*0.9
ax2 = fig.add_axes([left, bottom, width, height], projection=proj)
ax2.set_extent([105, 125, 0, 25], crs=proj)  # 设置南海区域范围

# 添加海洋和海岸线
ax2.add_feature(cfeature.OCEAN, facecolor='lightblue')
ax2.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=0.5)

# 添加矩形框边界并设置背景
ax2.spines['geo'].set_edgecolor('grey')  # 小地图边框颜色
ax2.spines['geo'].set_linewidth(1.0)
ax2.set_facecolor('#F5F5F5')  # 与小地图背景色一致

# 在南海小地图上显示相同的SM数据
imshow2 = ax2.imshow(SM, cmap=cmap, extent=[XLON.min(), XLON.max(), XLAT.min(), XLAT.max()], 
           origin='lower', alpha=0.9)
# ax2.imshow(SM, cmap=cmap, extent=[XLON.min(), XLON.max(), XLAT.min(), XLAT.max()], 
#            origin='lower', alpha=0.9)
imshow2.set_clim(0.0, 0.4)
# 添加比例尺标记
from matplotlib.font_manager import FontProperties
shp_add=r'/home/yfdong/data/map/SouthChinaSea/bou2_4l.shp'
ax2.add_geometries(Reader(shp_add).geometries(),ccrs.PlateCarree(),facecolor='Lightblue',edgecolor='k',linewidth=0.4)
# -----------------------------主图调整------------------------------------
# 确保主图数据可见 (已有代码保留)
imshows = ax.imshow(SM[:], cmap=cmap, extent=[XLON.min(), XLON.max(), XLAT.min(), XLAT.max()], 
                    origin='lower')
imshows.set_clim(0.0, 0.4)
plt.show()